In [1]:
import pandas as pd

In [4]:
nvoc_df = pd.read_csv('/aihub/data/nvocdata_final_20200720.csv', encoding = 'UTF-8')

In [5]:
nvoc_df

,Unnamed: 0,상담대분류,상담소분류,상담일자,상품구분,영업단,급지,해지일,AS신청유형,AS원인유형,AS처리유형,AS완료일자,AS처리시간,cnslt_rscs_diff,as_rscs_diff,rscs_yn
0,0,1,9,2016-01-08,0,1,4,2016-10-14,15,62,57,2016-01-08,28,280,280.0,False
1,1,1,9,2016-01-09,0,0,2,2017-04-17,25,62,57,2016-01-09,24,464,464.0,False
2,2,1,9,2016-01-09,0,0,2,2017-04-17,25,86,40,2014-02-18,24,464,1154.0,False
3,3,1,9,2016-01-09,0,0,2,2017-04-17,25,0,1,2016-01-08,1,464,465.0,False
4,4,1,9,2016-01-09,0,0,2,2017-04-17,13,62,57,2015-11-30,25,464,504.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117,2154,1,0,2020-06-10,0,1,2,2020-06-15,25,81,76,2018-08-31,2,5,654.0,True
2118,2155,1,0,2020-06-10,0,1,2,2020-06-15,23,49,0,2016-07-29,39,5,1417.0,True
2119,2156,1,0,2020-06-10,0,1,2,2020-06-15,18,62,57,2017-11-01,0,5,957.0,True
2120,2157,1,0,2020-06-10,0,1,2,2020-06-15,15,62,57,2015-11-12,0,5,1677.0,True
